# Text document Classification (Dataset)

https://www.kaggle.com/datasets/jensenbaxter/10dataset-text-document-classification

*   10 clases
*   1000 muestras de entrenamiento

In [3]:
import zipfile
with zipfile.ZipFile('/content/Dataset_text.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Dataset_text/')

# GloVe word embeddings file

In [5]:

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-10-04 16:53:20--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-10-04 16:53:20--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-10-04 16:53:20--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [6]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ")
    embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


In [10]:
from tensorflow import keras
batch_size = 32

In [ ]:
"""
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(2023).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)

In [11]:
train_ds = keras.utils.text_dataset_from_directory(
    "/content/Dataset_text",
    batch_size=batch_size,
    validation_split=0.2,
    label_mode="categorical",
    seed = 2023,
    subset="training",
)

Found 1000 files belonging to 10 classes.
Using 800 files for training.


In [12]:
val_ds = keras.utils.text_dataset_from_directory(
    "/content/Dataset_text",
    batch_size=batch_size,
    validation_split=0.2,
    label_mode="categorical",
    seed = 2023,
    subset="validation",
)

Found 1000 files belonging to 10 classes.
Using 200 files for validation.


In [ ]:
"""
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=batch_size
)
"""

'\ntest_ds = keras.utils.text_dataset_from_directory(\n    "aclImdb/test",\n    batch_size=batch_size\n)\n'

In [13]:
# Visualizar características del primer lote (batch)
for inputs, targets in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32, 10)
targets.dtype: <dtype: 'float32'>
inputs[0]: tf.Tensor(b'Toward Armistice\r\nBy the fall of 1918, the Central Powers were unraveling on all fronts.\r\n\r\nDespite the Turkish victory at Gallipoli, later defeats by invading forces and an Arab revolt had combined to destroy the Ottoman economy and devastate its land, and the Turks signed a treaty with the Allies in late October 1918.\r\n\r\nAustria-Hungary, dissolving from within due to growing nationalist movements among its diverse population, reached an armistice on November 4. Facing dwindling resources on the battlefield, discontent on the homefront and the surrender of its allies, Germany was finally forced to seek an armistice on November 11, 1918, ending World War I.', shape=(), dtype=string)
targets[0]: tf.Tensor([0. 0. 0. 0. 1. 0. 0. 0. 0. 0.], shape=(10,), dtype=float32)


# Data preparation



In [28]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# No se utilizan ngrams (none)
max_length = 600
max_tokens = 20000
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,    # Limitar la entrada a 600 palabras
)

# Extraer solo el texto (features) de los datos de entrada para calcular el vocabulario
text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorization.adapt(text_only_train_ds)

"""
int_train_ds = train_ds.map(
    lambda x, y: (tf.one_hot(text_vectorization(x), max_tokens), y),
    num_parallel_calls=4)

int_val_ds = val_ds.map(
    lambda x, y: (tf.one_hot(text_vectorization(x), max_tokens), y),
    num_parallel_calls=4)


int_test_ds = test_ds.map(
    lambda x, y: (tf.one_hot(text_vectorization(x), max_tokens), y),
    num_parallel_calls=4)
"""



int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
"""
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
"""

'\nint_test_ds = test_ds.map(\n    lambda x, y: (text_vectorization(x), y),\n    num_parallel_calls=4)\n'

# GloVe word-embeddings matrix

In [29]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
  if i < max_tokens:
    embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

# Modelo

In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input, Dropout, Bidirectional

embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
    )

In [31]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(10, activation="softmax")(x)

model_Embedded3 = keras.Model(inputs, outputs)

model_Embedded3.compile(optimizer="rmsprop",
                        loss="categorical_crossentropy",
                        metrics=["accuracy"]
                        )

model_Embedded3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_2 (Bidirecti  (None, 64)                34048     
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
Total params: 2034698 (7.76 MB)
Trainable params: 34698 (135.54 KB)
Non-trainable params: 2000000 (7.63 MB)
_________________

# Entrenamiento

In [32]:
# Utilización de caché
model_Embedded3.fit(
    int_train_ds,
    validation_data=int_val_ds,
    epochs=20,
    #callbacks=callbacks
    )

Epoch 1/20
25/25 [==============================] - 15s 190ms/step - loss: 2.2031 - accuracy: 0.2100 - val_loss: 1.9449 - val_accuracy: 0.5800
Epoch 2/20
25/25 [==============================] - 2s 94ms/step - loss: 1.7952 - accuracy: 0.4600 - val_loss: 1.4693 - val_accuracy: 0.7150
Epoch 3/20
25/25 [==============================] - 3s 108ms/step - loss: 1.4433 - accuracy: 0.5913 - val_loss: 1.1423 - val_accuracy: 0.7350
Epoch 4/20
25/25 [==============================] - 2s 96ms/step - loss: 1.1599 - accuracy: 0.6700 - val_loss: 0.9292 - val_accuracy: 0.7600
Epoch 5/20
25/25 [==============================] - 2s 96ms/step - loss: 0.9755 - accuracy: 0.7275 - val_loss: 0.7227 - val_accuracy: 0.8350
Epoch 6/20
25/25 [==============================] - 2s 90ms/step - loss: 0.9189 - accuracy: 0.7237 - val_loss: 0.6147 - val_accuracy: 0.8550
Epoch 7/20
25/25 [==============================] - 2s 95ms/step - loss: 0.7399 - accuracy: 0.8012 - val_loss: 0.6377 - val_accuracy: 0.8150
Epoch 8/20

#Evaluación del modelo

Sobre datos test